In [1]:
%cd ..

/home/ead/iking5/code/bitcoin_fraud_detection


In [2]:
import torch
from resources.constants import LICIT, ILLICIT
graphs = torch.load('resources/graphs.pt')

/home/ead/iking5/miniconda3/envs/btc/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [40]:
def hops_to_label(g, nid, k_hops, target):
    hops = 0
    explored = set() 
    domain = set([nid])

    while hops < k_hops:
        hops += 1
        new_domain = set() 
        for n in domain:
            if n in explored:
                continue 

            neighbors = g.edge_index[1, g.edge_index[0]==n]
            if target in g.y[neighbors]:
                return hops 
            
            explored.add(n)
            [new_domain.add(n.item()) for n in neighbors]
        
        domain = new_domain
    
    # Using 0 to represent no illicit neighbors in neighborhood
    return 0

from tqdm import tqdm 
def find_all_distances(i, g, src=LICIT, dst=ILLICIT, k_hops=3, aggr=False):
    print(f'\r{i}', end='')
    licit = (g.y == src).nonzero().squeeze(-1)
    dists = [
        hops_to_label(g, nid, k_hops, dst) 
        for nid in licit
    ]

    t = torch.tensor(dists).float()

    if aggr:
        nz = t.nonzero()
        return {
            'avg_connected': nz.size(0)/t.size(0),
            'avg_distance': t[nz].mean().item()
        }
    else:
        return t 

In [44]:
from joblib import Parallel, delayed
import pandas as pd  
l2i = pd.DataFrame(
    Parallel(prefer='processes', n_jobs=16)(
        delayed(find_all_distances)(i,g,aggr=True) for i,g in enumerate(graphs)
    )
)
i2i = pd.DataFrame(
    Parallel(prefer='processes', n_jobs=16)(
        delayed(find_all_distances)(i,g,aggr=True, src=ILLICIT, dst=ILLICIT) for i,g in enumerate(graphs)
    )
)
l2l = pd.DataFrame(
    Parallel(prefer='processes', n_jobs=16)(
        delayed(find_all_distances)(i,g,aggr=True,dst=LICIT) for i,g in enumerate(graphs)
    )
)
i2l = pd.DataFrame(
    Parallel(prefer='processes', n_jobs=16)(
        delayed(find_all_distances)(i,g,aggr=True, src=ILLICIT, dst=LICIT) for i,g in enumerate(graphs)
    )
)

48

In [48]:
df = pd.DataFrame()
for name,d in [('l2i',l2i), ('i2i',i2i), ('l2l',l2l), ('i2l',i2l)]:
    df[name+'-avg_connected'] = d['avg_connected']
    df[name+'-avg_distance'] = d['avg_distance']

df = df.fillna(0)

,avg_connected,avg_distance
0,0.000000,NaN
1,0.000000,NaN
2,0.000000,NaN
3,0.133333,1.000000
4,0.000000,NaN
5,0.000000,NaN
6,0.166667,1.000000
7,0.164179,1.090909
8,0.116935,1.413793
9,0.166667,1.000000
